In [1]:
import numpy as np
import gpt as g

In [2]:
g.default.set_verbose(False)
g.default.set_verbose("step_size", False)

#################################
### General global parameters ###
#################################
L1, L2, L3 = 24, 24, 24
T   = 48 
Ntherm = 100        # thermalization trajectories
therm_step = 10
Ntraj  = 10
tau = 2.0           # length of MD trajectory
Ns  = 6             # integration steps of HMC
seed = "12345"
beta = g.default.get_float("-- beta", 5.96)

In [8]:
grid = g.grid([L1, L2, L3, T], g.double)
rng = g.random(seed)

U = g.qcd.gauge.unit(grid)
rng.normal_element(U)

# links conjugate momenta
pi = g.group.cartesian(U)

GPT :     134.388389 s : Initializing gpt.random(12345,vectorized_ranlux24_389_64) took 0.00054884 s


In [10]:
for link in U:
    link[:] = 0.0

In [12]:
U[0][0,0,0,0]

tensor([[0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j]],ot_matrix_su_n_fundamental_group(3))

In [17]:
def plaquette_slice(U, dim):
        Nd = len(U)
        vol = float(U[0].grid.fsites)
        ndim = U[0].otype.shape[0]
        tr = g.complex(grid)
        tr[:] = 0.0
        for mu in range(Nd):
                for nu in range(mu):
                        tr += g.trace(
                                        U[mu] * g.cshift(U[nu], mu, 1) * g.adj(g.cshift(U[mu], nu, 1)) * g.adj(U[nu])
                                )
        tr = np.array(g.slice(tr, 3)).real
        return 2.0 * tr / vol / Nd / (Nd - 1) / ndim


GPT :    1744.180562 s : 0.04406449998522606
GPT :    1744.392021 s : 0.04406449998522605


In [18]:
g.message(g.qcd.gauge.energy_density(U, field=False))

GPT :    2536.459233 s : (2.296378364327372-1.1609820748862098e-37j)


In [21]:
g.message(np.sum(np.array(g.slice(g.qcd.gauge.energy_density(U,         field=True), 3)).real)/U[0].grid.fsites)

GPT :    2679.479622 s : 2.296378364327373


In [4]:
#########################################
### General informations for Log file ###
#########################################
g.message(f"  Lattice  =  {grid.fdimensions}")

a0 = g.qcd.scalar.action.mass_term() # action for conjugate momenta
a1 = g.qcd.gauge.action.wilson(beta) # Wilson action

g.message()
g.message("  Actions: ")
g.message(f"   -- {a0.__name__}")
g.message(f"   -- {a1.__name__} with coupling beta = {beta}")

GPT :      68.332059 s :   Lattice  =  [24, 24, 24, 48]
GPT :      68.333077 s : 
GPT :      68.334406 s :   Actions: 
GPT :      68.334811 s :    -- mass_term(1.0)
GPT :      68.335316 s :    -- wilson(5.96) with coupling beta = 5.96


In [5]:
####################################
### HMC setup: MD and Metropolis ###
####################################

# molecular dynamics
sympl = g.algorithms.integrator.symplectic

ip = sympl.update_p(pi, lambda: a1.gradient(U, U))
iq = sympl.update_q(U,  lambda: a0.gradient(pi, pi))

# integrator
mdint = sympl.OMF4(Ns, ip, iq)
g.message()
g.message(f"  Integration scheme:\n{mdint}")

# metropolis
metro = g.algorithms.markov.metropolis(rng)

# MD units
g.message(f"  tau = {tau} MD units")


def HMC(tau, pi):
    rng.normal_element(pi)
    accrej = metro(U)
    S_i = a1(U)
    H_i = a0(pi) + S_i
    mdint(tau) # MD evolution
    S_f = a1(U) 
    H_f = a0(pi) + S_f 
    return [accrej(H_f, H_i), H_i, H_f]

GPT :      68.349658 s : 
GPT :      68.350492 s :   Integration scheme:
                       :  - Level 0 omf4 steps=6
                       :  - Level 1 euler steps=1
GPT :      68.351135 s :   tau = 2.0 MD units


In [6]:
######################
### Thermalization ###
######################
g.message()
g.message(f"--------------------------------------------------------------------------------------------")
g.message(f"-------------------------------- Starting thermalization... --------------------------------")
g.message(f"--------------------------------------------------------------------------------------------")
g.message()
g.message()

timer = g.timer("HMC")
for i in range(therm_step):
    h    = []
    plaq = []
    dt = 0.0
    for j in range(Ntherm // therm_step):
        timer("trajectory")
        h    += [HMC(tau, pi)]
        plaq += [g.qcd.gauge.plaquette(U)]
        timer()
        dt   += timer.time['trajectory'].dt_last
    h = np.array(h) 
    plaq = np.array(plaq)
    g.message()
    g.message(f"  ----- {(i+1)*(Ntherm//therm_step)/Ntherm*100:.0f}% of Thermalization completed  :  avg. time single traj. = {dt / (Ntherm // therm_step):.3f} secs -----  ")
    g.message(f"  < Plaq > = {np.mean(plaq):.4f},  Acceptance = {np.mean(h[:, 0])*100:.0f}%,  < |dH| > = {np.mean(np.abs(h[:, 2] - h[:, 1])):.3e}  ")
    g.message()

GPT :      82.350219 s : 
GPT :      82.351069 s : --------------------------------------------------------------------------------------------
GPT :      82.351747 s : -------------------------------- Starting thermalization... --------------------------------
GPT :      82.352437 s : --------------------------------------------------------------------------------------------
GPT :      82.353155 s : 
GPT :      82.353776 s : 


KeyboardInterrupt: 